In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


class SagittalStructure(object):
    """SagittalStructure describes the static portions of the model.
    They describe the locations of bones, etc.
    They are determined through analysis of just one MRI image: vowel a; and 
    assumed to stay constant
    """

    lens = [24, 24, 7, 6, 6, 6, 6, 6, 6, 6, 6, 20, 8, 6, 4, 8, 8, 8, 8]
    cutoffs = np.cumsum(lens)
    labels = ['hw1', 'hw2', 'hw3', 'hw4', 'hw5', 'hw6',
              'hw7', 'hw8', 'hw9', 'hw10', 'hw11',
              'sc1', 'sc2', 'sc3', 'sc4',
              'ok1', 'ok2', 'ok3',
              'na']

    def __init__(self, data):
        self.data = data
        last = 0
        for label, cutoff in zip(self.labels, self.cutoffs):
            setattr(self, label, self.data[last:cutoff])
            last = cutoff


class SagittalContours(object):
    """Dynamic portions of the model"""
    lens = [10, 10, 26, 4, 8, 16, 16, 34, 40]
    cutoffs = np.cumsum(lens)
    labels = ['jaw1', 'jaw2', 'jaw3',
              'larynx1', 'larynx2',
              'lips1', 'lips2',
              'velum', 'tongue']

    def __init__(self, data):
        if data.ndim == 2:
            data = np.reshape(data, (data.shape[0], 1, data.shape[1]))
        self.data = data
        assert self.data.shape[0] == sum(self.lens), self.data.shape
        assert self.data.shape[2] == 2, self.data.shape
        last = 0
        for label, cutoff in zip(self.labels, self.cutoffs):
            setattr(self, label, self.data[last:cutoff])
            last = cutoff

    @classmethod
    def from_articulators(cls, art):
        # jaw + lips
        jaw = np.arange(cls.cutoffs[2], dtype=int)
        lips = np.arange(cls.cutoffs[4], cls.cutoffs[6], dtype=int)
        jaw_and_lips = np.hstack([jaw, lips])

        # --- 1. Vocalic articulation
        # The main operation here is interpolating between the three extremal
        # vocalic states: a, i and u:
        #   iuFac(ii * iiFac + uu * uuFac) + aa * aaFac
        iiFac = np.tile(art.iiFac[..., np.newaxis], (sum(cls.lens), 1, 2))
        uuFac = np.tile(art.uuFac[..., np.newaxis], (sum(cls.lens), 1, 2))
        iiFac[jaw_and_lips] = np.hstack([art.iiFacLips.T, art.iiFacLips.T])
        uuFac[jaw_and_lips] = np.hstack([art.uuFacLips.T, art.uuFacLips.T])

        iuFac = np.hstack([art.iuFac.T, art.iuFac.T])
        aaFac = np.hstack([art.aaFac.T, art.aaFac.T])
        ret = cls(data=iuFac * (cls.ii.data * iiFac + cls.uu.data * uuFac)
                  + cls.aa.data * aaFac)

        # save vocalic pos of lower jaw
        low_teeth = np.array(ret.jaw1)

        # --- 2. Velum articulation
        # The velum position is an interpolation between the state thus far
        # and the 'n' contour.
        velFac = np.hstack([art.velFac.T, art.velFac.T])
        ret.velum *= 1.0 - velFac
        ret.velum += cls.nn.velum * velFac

        # --- 3. Consonantal lip articulation.
        # This involves interpolating between the state thus far and the
        # 'm' contour. This is used for consonants such as /p/, /b/, /m/.
        lipClosFac = np.hstack([art.lipClosFac.T, art.lipClosFac.T])
        #ret.data[jaw_and_lips] *= 1.0 - lipClosFac
        #ret.data[jaw_and_lips] += cls.mm.data[jaw_and_lips] * lipClosFac
        ret.data[lips] *= 1.0 - lipClosFac
        ret.data[lips] += cls.mm.data[lips] * lipClosFac
        ret.data[jaw] *= 1.0 - lipClosFac*0.66
        ret.data[jaw] += cls.mm.data[jaw] * lipClosFac*0.66
        # smooth connection of lips with lower jaw
        ret.lips2[0] = ret.jaw3[0]
        ret.lips2[-1] = ret.jaw1[-5]  

        # --- 4. Co-elevation of tongue and lower jaw
        # This is a special case; if the consonantal lip articulation causes
        # an elevation of the jaw, the tongue also needs to be raised.
        low_teeth = ret.jaw1 - low_teeth
        # fully elevate only the front part of tongue
        ret.tongue[-5:] += low_teeth[-5:]
        # linear increase of elevation for the front-middle part
        ramp = np.linspace(0, 1, 10)[:, np.newaxis, np.newaxis]
        ret.tongue[-15:-5] += low_teeth * ramp

        # --- 5. Consonantal tongue dorsum articulation.
        # This involves creating a new 'k' extremal contour, and then
        # interpolating between the current state and this new contour
        # based on the toDors3 parameter.
        # This is used for consonants such as /k/, /g/.
        iuFacClipped = np.hstack([art.iuFacClipped.T, art.iuFacClipped.T])
        iiFacClipped = np.hstack([art.iiFacClipped.T, art.iiFacClipped.T])
        uuFacClipped = np.hstack([art.uuFacClipped.T, art.uuFacClipped.T])
        aaFacClipped = np.hstack([art.aaFacClipped.T, art.aaFacClipped.T])
        kk_tongue = (iuFacClipped * (cls.ii.tongue * iiFacClipped
                                     + cls.uu.tongue * uuFacClipped)
                     + cls.aa.tongue * aaFacClipped)
        fac = 0.5
        toDorsClosFac = np.hstack([art.toDorsClosFac.T, art.toDorsClosFac.T])
        ramp = np.hstack([np.linspace(0, 1, 13),  # ramp up near epiglottis
                          np.ones(6),  # full effect for a while
                          np.linspace(1, fac, 9)])  # ramp down including tongue tip to fac=0.5
        ramp = ramp[:, np.newaxis, np.newaxis]
        ret.tongue[12:40] *= 1.0 - toDorsClosFac * ramp
        ret.tongue[12:40] += kk_tongue[12:40] * toDorsClosFac * ramp
        # co-articulation of jaw mowement: jaw1, jaw2, jaw3, lips2 
        kk_jaw1 = (iuFacClipped * (cls.ii.jaw1 * iiFacClipped
                                     + cls.uu.jaw1 * uuFacClipped)
                     + cls.aa.jaw1 * aaFacClipped)
        ret.jaw1[:] *= 1.0 - toDorsClosFac*fac
        ret.jaw1[:] += kk_jaw1[:] * toDorsClosFac*fac
        kk_jaw2 = (iuFacClipped * (cls.ii.jaw2 * iiFacClipped
                                     + cls.uu.jaw2 * uuFacClipped)
                     + cls.aa.jaw2 * aaFacClipped)
        ret.jaw2[:] *= 1.0 - toDorsClosFac*fac
        ret.jaw2[:] += kk_jaw2[:] * toDorsClosFac*fac
        kk_jaw3 = (iuFacClipped * (cls.ii.jaw3 * iiFacClipped
                                     + cls.uu.jaw3 * uuFacClipped)
                     + cls.aa.jaw3 * aaFacClipped)
        ret.jaw3[:] *= 1.0 - toDorsClosFac*fac
        ret.jaw3[:] += kk_jaw3[:] * toDorsClosFac*fac
        kk_lips2 = (iuFacClipped * (cls.ii.lips2 * iiFacClipped
                                     + cls.uu.lips2 * uuFacClipped)
                     + cls.aa.lips2 * aaFacClipped)
        ret.lips2[:] *= 1.0 - toDorsClosFac*fac
        ret.lips2[:] += kk_lips2[:] * toDorsClosFac*fac

        # --- 6. Consonantal tongue tip articulation.
        # This involves interpolating between the state thus far and the
        # 'n' contour. This is used for consonants such as /t/, /d/, /n/, /l/.
        begin, end = 15, 37
        start, mid = 32, 35
        fac = 0.5
        # not used currently
        high_toDors1_ramp = np.hstack([np.zeros(start - begin),
                                       np.linspace(0, 1, mid - start),
                                       np.ones(end - mid),
                                       np.linspace(1, 0, 40 - end)])
        start, mid = 15, 30
        # active!
        low_toDors1_ramp = np.hstack([np.zeros(start - begin),
                                      np.linspace(0, 1, mid - start),
                                      np.ones(end - mid),
                                      np.linspace(1, fac, 40 - end)])
        ramp = np.tile(low_toDors1_ramp[:, np.newaxis],
                       (1, art.toDors1.shape[1]))
        # do not use high_ramp in moment!
        ramp[:, art.toDors1[0] > 1000] = high_toDors1_ramp[:, np.newaxis]
        ramp = ramp[:, :, np.newaxis]
        toTipClosFac = np.hstack([art.toTipClosFac.T, art.toTipClosFac.T])
        ret.tongue[begin:] *= 1.0 - toTipClosFac * ramp
        ret.tongue[begin:] += cls.nn.tongue[start:] * toTipClosFac * ramp
        # co-articulation of jaw mowement: jaw1, jaw2, jaw3, lips2 
        ret.jaw1[:] *= 1.0 - toTipClosFac*fac
        ret.jaw1[:] += cls.nn.jaw1[:] * toTipClosFac*fac
        ret.jaw2[:] *= 1.0 - toTipClosFac*fac
        ret.jaw2[:] += cls.nn.jaw2[:] * toTipClosFac*fac
        ret.jaw3[:] *= 1.0 - toTipClosFac*fac
        ret.jaw3[:] += cls.nn.jaw3[:] * toTipClosFac*fac
        ret.lips2[:] *= 1.0 - toTipClosFac*fac
        ret.lips2[:] += cls.nn.lips2[:] * toTipClosFac*fac
        
        return ret


class Articulators(object):
    lens = [1, 1, 1, 1, 1, 1, 1, 1]
    cutoffs = np.cumsum(lens)
    labels = ['toDors2', 'toDors1', 'toDors3',  # vocalic front-back, vocalic hgih-low, tongue voc_closure
              'lips1', 'lips2',  # mouth voc-closed, lips unrounded-rounded 
              'toTip1', 'toTip2',  # tongue tip voc-high, front-back
              'velum']   # velum high-low 
    ranges = [(0, 1000),
              (-1000, 1000),
              (-1000, 1000),
              (0, 1000),
              (0, 1000),
              (0, 1000),
              (-1000, 1000),
              (-1000, 0)]

    def __init__(self, data):
        self.data = data[:, np.newaxis] if data.ndim == 1 else data
        assert self.data.shape[0] == sum(self.lens), self.data.shape
        last = 0
        for label, cutoff, rnge in zip(self.labels, self.cutoffs, self.ranges):
            if np.any(self.data[last:cutoff] < rnge[0]):
                print("%s value too low; clipping" % label)
            if np.any(self.data[last:cutoff] > rnge[1]):
                print("%s value too high; clipping" % label)
            setattr(self, label,
                    np.clip(self.data[last:cutoff], rnge[0], rnge[1]))
            last = cutoff

    def __setattr__(self, key, value):
        if hasattr(self, key) and key in self.labels:
            getattr(self, key)[...] = value
        else:
            super(Articulators, self).__setattr__(key, value)

    @property
    def iiFac(self):
        return (self.toDors2 + 1000.0) / 2000.0

    @property
    def uuFac(self):
        return 1.0 - self.iiFac

    @property
    def iiFacClipped(self):
        return (np.clip(self.toDors2, -500.0, 500.0) + 1000.0) / 2000.0

    @property
    def uuFacClipped(self):
        return 1.0 - self.iiFacClipped

    @property
    def iuFac(self):
        return (self.toDors1 + 1000.0) / 2000.0

    @property
    def aaFac(self):
        return 1.0 - self.iuFac

    @property
    def iuFacClipped(self):
        return np.ones_like(self.iiFac) * ((1400.0 + 1000.0) / 2000.0)

    @property
    def aaFacClipped(self):
        return 1.0 - self.iuFacClipped

    @property
    def uuFacLips(self):
        return self.lips2 / 1000.0

    @property
    def iiFacLips(self):
        return 1.0 - self.uuFacLips

    @property
    def lipClosFac(self):
        return self.lips1 / 1000.0

    @property
    def toTipClosFac(self):
        return self.toTip1 / 1000.0

    @property
    def toDorsClosFac(self):
        return self.toDors3 / 1000.0

    @property
    def velFac(self):
        return self.velum / 1000.0


def plot_sagittal(structure, contour, highlight='tongue', ax=None):
    if ax is None:
        ax = plt.gca()
    ax.set_aspect(1.0)
    ax.set_xticks(())
    ax.set_yticks(())

    for label in structure.labels:
        el = getattr(structure, label)
        ax.plot(el.T[0], el.T[1], color='b', linestyle='-', marker='')  # *

    c_lines = []
    for label in contour.labels:
        color = 'r' if label == highlight else 'b'
        el = getattr(contour, label)
        c_lines.append(ax.plot(el[:, 0, 0], el[:, 0, 1], color=color, linestyle='-', marker='')[0])  # *
    return c_lines

def update_sagittal(c_lines, contour, i):
    for lines, label in zip(c_lines, contour.labels):
        el = getattr(contour, label)
        lines.set_data(el[:, i, 0], el[:, i, 1])
    return c_lines


def animate_sagittal(structure, contours, highlight='tongue', ax=None):
    if ax is None:
        ax = plt.gca()

    c_lines = plot_sagittal(structure, contours, highlight, ax=ax)

    def init():
        for lines in c_lines:
            lines.set_data([], [])
        return c_lines

    def update(i):
        return update_sagittal(c_lines, contours, i)

    return animation.FuncAnimation(plt.gcf(), update,
                                   init_func=init,
                                   frames=contours.data.shape[1],
                                   interval=15,
                                   blit=True)


#root = os.path.dirname(os.path.realpath(__file__))
structure = SagittalStructure(np.loadtxt('data_sagi/structure.txt'))
SagittalContours.ii = SagittalContours(np.loadtxt('data_sagi/ii.txt'))
SagittalContours.aa = SagittalContours(np.loadtxt('data_sagi/aa.txt'))
SagittalContours.uu = SagittalContours(np.loadtxt('data_sagi/uu.txt'))
SagittalContours.mm = SagittalContours(np.loadtxt('data_sagi/mm.txt'))
SagittalContours.nn = SagittalContours(np.loadtxt('data_sagi/nn.txt'))
SagittalContours.sh = SagittalContours(np.loadtxt('data_sagi/sh.txt'))
SagittalContours.th = SagittalContours(np.loadtxt('data_sagi/th.txt'))
SagittalContours.ff = SagittalContours(np.loadtxt('data_sagi/ff.txt'))

In [ ]:
# zum betrachten der Berechnungen (statisch): 
def show_sagi(articulators, highlight="tongue"):
    plt.figure()
    fig = plt.figure(figsize=(9,9))
    ax = plt.subplot(1,1,1)
    art_contour = SagittalContours.from_articulators(articulators)
    plot_sagittal(structure, art_contour, ax=ax, highlight=highlight)

# Überlagere zwei Konturen: artic1 und artic2
def show_two_sagis(artic1, artic2, highlight="none"):
    plt.figure()
    fig = plt.figure(figsize=(9,9))
    ax = plt.subplot(1,1,1)
    artic1_contour = SagittalContours.from_articulators(artic1)
    plot_sagittal(structure, artic1_contour, ax=ax, highlight=highlight)
    artic2_contour = SagittalContours.from_articulators(artic2)
    plot_sagittal(structure, artic2_contour, ax=ax, highlight=highlight)
    
# zum Betrachten auch von BasisKonturen
def compare(contour, articulators, highlight="tongue"):
    plt.figure()
    ax = plt.subplot(121)
    plot_sagittal(structure, contour, ax=ax, highlight=highlight)
    ax = plt.subplot(122)
    art_contour = SagittalContours.from_articulators(articulators)
    plot_sagittal(structure, art_contour, ax=ax, highlight=highlight)
    
# setze einige Parameter direkt wie Vokale
def set_vowel(art, vowel="0"):
    if vowel == 'a':
        art.toDors1 = -1000
    elif vowel == 'i':
        art.toDors1 = 1000
        art.toDors2 = 1000
    elif vowel == 'u':
        art.toDors1 = 1000
        art.toDors2 = -1000
        art.lips2 = 1000
    elif vowel =='neutral':   
        art.toDors1 = 0
        art.toDors2 = 0
    elif vowel =='neut_back':   
        art.toDors1 = 0
        art.toDors2 = -1000
    elif vowel =='neut_front':   
        art.toDors1 = 0
        art.toDors2 = -1000
    else:
        art.toDors1 = 0
        art.toDors2 = 0

In [ ]:
# plot overlay of two contours: /a/-/i/
art1 = Articulators(np.zeros(8))
art1.toDors1 = 1000
art1.toDors2 = 1000
art1.velum = 300
art2 = Articulators(np.zeros(8))
art2.toDors1 = -500
art2.toDors2 = 1000
art2.velum = 0
show_two_sagis(art1, art2, highlight="none")

In [ ]:
# plot overlay of two contours: /a/-/u/(lips unrounded)
art1 = Articulators(np.zeros(8))
art1.toDors1 = 1000
art1.toDors2 = 1000
art1.velum = 0
art2 = Articulators(np.zeros(8))
art2.toDors1 = 1000
art2.toDors2 = -1000
art2.velum = 0
show_two_sagis(art1, art2, highlight="none")

In [ ]:
# plot overlay of two contours: /i/ (lips rounded-spread)
art1 = Articulators(np.zeros(8))
art1.toDors1 = 1000
art1.toDors2 = 1000
art1.velum = 0
art2 = Articulators(np.zeros(8))
art2.toDors1 = 1000
art2.toDors2 = 1000
art2.lips2 = 1000
art2.velum = 0
show_two_sagis(art1, art2, highlight="none")

In [ ]:
# plot overlay of two contours: /a/-/k/
art1 = Articulators(np.zeros(8))
art1.toDors1 = -500
art1.toDors2 = -500
art1.velum = -100
art2 = Articulators(np.zeros(8))
art2.toDors1 = -500
art2.toDors2 = -500
art2.toDors3 = 950
art2.velum = -100
show_two_sagis(art1, art2, highlight="none")

In [ ]:
# plot overlay of two contours: /a/-/t/
art1 = Articulators(np.zeros(8))
art1.toDors1 = -500
art1.toDors2 = 0
art1.velum = 0
art2 = Articulators(np.zeros(8))
art2.toDors1 = -500
art2.toDors2 = 0
art2.toTip1 = 1000
art2.velum = 0
show_two_sagis(art1, art2, highlight="none")

In [ ]:
# plot overlay of two contours: /a/-/p/
art1 = Articulators(np.zeros(8))
art1.toDors1 = -500
art1.toDors2 = 0
art1.velum = 0
art2 = Articulators(np.zeros(8))
art2.toDors1 = -500
art2.toDors2 = 0
art2.lips1 = 900
art2.velum = 0
show_two_sagis(art1, art2, highlight="none")

In [ ]:
# plot overlay of two contours: /schwa/-/schwa nas/
art1 = Articulators(np.zeros(8))
art1.toDors1 = 0
art1.toDors2 = 0
art1.velum = 0
art2 = Articulators(np.zeros(8))
art2.toDors1 = 0
art2.toDors2 = 0
art2.velum = 1000
show_two_sagis(art1, art2, highlight="none")